# TSA Chapter 9: Electricity Demand

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch9_electricity_demand/TSA_ch9_electricity_demand.ipynb)

This notebook generates the simulated hourly electricity demand visualization with daily, weekly, and annual seasonal patterns.

In [ ]:
!pip install numpy pandas matplotlib -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Color scheme
BLUE    = '#1A3A6E'
RED     = '#DC3545'
GREEN   = '#2E7D32'
ORANGE  = '#E67E22'
GRAY    = '#666666'
PURPLE  = '#8E44AD'

# Transparent backgrounds
plt.rcParams['figure.facecolor'] = 'none'
plt.rcParams['axes.facecolor'] = 'none'
plt.rcParams['savefig.facecolor'] = 'none'
plt.rcParams['savefig.transparent'] = True

# No top/right spines
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False

# General styling
plt.rcParams['axes.grid'] = False
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Helvetica', 'Arial', 'DejaVu Sans']
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 10
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['xtick.labelsize'] = 9
plt.rcParams['ytick.labelsize'] = 9
plt.rcParams['legend.fontsize'] = 9
plt.rcParams['legend.facecolor'] = 'none'
plt.rcParams['legend.framealpha'] = 0
plt.rcParams['axes.linewidth'] = 0.6
plt.rcParams['lines.linewidth'] = 1.0


def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    try:
        charts_path = os.path.join('..', '..', 'charts', name)
        fig.savefig(f'{charts_path}.pdf', bbox_inches='tight', transparent=True, dpi=150)
        fig.savefig(f'{charts_path}.png', bbox_inches='tight', transparent=True, dpi=150)
    except Exception:
        pass
    print(f'Saved: {name}.pdf + .png')


def legend_outside(ax, ncol=3):
    """Place legend outside bottom of plot."""
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.18), ncol=ncol, frameon=False)

In [ ]:
# Chart: ch9_electricity_demand
# Simulated hourly electricity demand with daily+weekly+annual patterns
np.random.seed(123)
n_hours = 24 * 7 * 4  # 4 weeks of hourly data
t_h = np.arange(n_hours)
hours = t_h % 24
days = t_h // 24

# Daily pattern: peak at noon and evening
daily_pattern = 20 * np.exp(-((hours - 12)**2) / 20) + 15 * np.exp(-((hours - 19)**2) / 10)
# Weekly pattern: lower on weekends
weekly_pattern = np.where(days % 7 >= 5, -10, 0)
# Base load
base = 100
demand = base + daily_pattern + weekly_pattern + np.random.normal(0, 3, n_hours)

fig, axes = plt.subplots(2, 1, figsize=(14, 7))

# Full 4 weeks
axes[0].plot(t_h, demand, color=BLUE, linewidth=0.5, label='Electricity Demand (MW)')
# Shade weekends
for w in range(4):
    start = (w * 7 + 5) * 24
    end = (w * 7 + 7) * 24
    if end <= n_hours:
        axes[0].axvspan(start, end, alpha=0.15, color=ORANGE, label='Weekend' if w == 0 else '')
axes[0].set_title('Hourly Electricity Demand (4 Weeks)', fontweight='bold')
axes[0].set_xlabel('Hour')
axes[0].set_ylabel('Demand (MW)')
legend_outside(axes[0], ncol=2)

# Zoom to 1 week
axes[1].plot(t_h[:168], demand[:168], color=RED, linewidth=0.8, label='Week 1')
day_labels = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
for d in range(7):
    axes[1].axvline(x=d*24, color=GRAY, linestyle=':', alpha=0.3)
    axes[1].text(d*24 + 12, axes[1].get_ylim()[0] if d < 2 else demand[:168].min()-2,
                 day_labels[d], ha='center', fontsize=8, color=GRAY)
axes[1].axvspan(5*24, 7*24, alpha=0.15, color=ORANGE, label='Weekend')
axes[1].set_title('Detail: One Week', fontweight='bold')
axes[1].set_xlabel('Hour')
axes[1].set_ylabel('Demand (MW)')
legend_outside(axes[1], ncol=2)

plt.tight_layout(h_pad=3.0)
save_chart(fig, 'ch9_electricity_demand')
plt.show()